In [1]:
import numpy as np
from PIL import Image
import os
from random import randint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint,LambdaCallback
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation


2024-04-25 10:24:20.473879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 10:24:20.474000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 10:24:20.611925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
lookup = {}
reverselookup = {}
count = 0

# Iterate through gesture directories
for gesture_dir in os.listdir('../input/leapgestrecog/leapGestRecog/00/'):
    if not gesture_dir.startswith('.'):  # Ignore hidden files/directories
        lookup[gesture_dir] = count
        reverselookup[count] = gesture_dir
        count += 1

# Print the lookup dictionary
print("Lookup Dictionary:")
print(lookup)

Lookup Dictionary:
{'02_l': 0, '04_fist_moved': 1, '09_c': 2, '10_down': 3, '06_index': 4, '08_palm_moved': 5, '07_ok': 6, '05_thumb': 7, '01_palm': 8, '03_fist': 9}


In [ ]:
x_data = []
y_data = []
datacount = 0

for i in range(10):  
    for j in os.listdir('../input/leapgestrecog/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'):  
            count = 0 
            for k in os.listdir('../input/leapgestrecog/leapGestRecog/0' + str(i) + '/' + j + '/'):
                
                img = Image.open('../input/leapgestrecog/leapGestRecog/0' + str(i) + '/' + j + '/' + k).convert('L')
               
                img = img.resize((320, 120))
                arr = np.array(img)
                x_data.append(arr)
                count += 1
            y_values = np.full((count, 1), lookup[j])
            y_data.extend(y_values)
            datacount += count

x_data = np.array(x_data, dtype='float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) 


In [ ]:
from random import randint
import matplotlib.pyplot as plt


num_images = 10
num_samples = len(x_data)

for _ in range(num_images):
    index = randint(0, num_samples - 1)  
    plt.imshow(x_data[index])  
    plt.title(reverselookup[y_data[index, 0]])  
    plt.show()

In [ ]:
# Preprocess data
x_data = x_data.reshape((x_data.shape[0], 120, 320, 1))
x_data.shape

In [ ]:
x_data /= 255
x_data[0]

In [ ]:
y_data = to_categorical(y_data)
y_data

In [ ]:
# Split data into training, validation, and test sets
x_train, x_further, y_train, y_further = train_test_split(x_data, y_data, test_size=0.2)
x_validate, x_test, y_validate, y_test = train_test_split(x_further, y_further, test_size=0.5)


In [ ]:
# Define model architecture
model = Sequential()
model.add(Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(120, 320, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Define the model
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(120, 320, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dense(10, activation="softmax"))


In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True)

# Define model checkpoint callback to save best model weights
model_checkpoint = ModelCheckpoint('best_model_weights.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train model with all callbacks
history = model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=1, 
                    validation_data=(x_validate, y_validate), 
                    callbacks=[early_stopping, model_checkpoint])

In [ ]:
import matplotlib.pyplot as plt


# Plot training history
plt.figure(figsize=(12, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate model on test data
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)